In [1]:
import pandas as pd 
import numpy as np 
import requests
import xlsxwriter
from scipy import stats

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

# Key stats of stocks

In [3]:
symbol = 'AAPL'
url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2143237763361,
 'week52high': 137.99,
 'week52low': 57.98,
 'week52change': 0.6229180833518945,
 'sharesOutstanding': 16854465075,
 'float': 0,
 'avg10Volume': 97501160,
 'avg30Volume': 113577635,
 'day200MovingAvg': 121.11,
 'day50MovingAvg': 130.43,
 'employees': 137203,
 'ttmEPS': 3.36,
 'ttmDividendRate': 0.8356770477054674,
 'dividendYield': 0.006608771436877497,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-28',
 'nextEarningsDate': '0',
 'peRatio': 38.57455202533555,
 'beta': 1.1662154741040929,
 'maxChangePercent': 49.36986268010966,
 'year5ChangePercent': 4.761728511252056,
 'year2ChangePercent': 2.3284436028197484,
 'year1ChangePercent': 0.6261039403145139,
 'ytdChangePercent': -0.04383901535880652,
 'month6ChangePercent': 0.3325947179800736,
 'month3ChangePercent': 0.07316887557694798,
 'month1ChangePercent': 0.004012780371855685,
 'day30ChangePercent': 0.003858066333610263,
 'day5ChangePercent': -0.014893370002610732}

# Split the symbols into sublists and use the Batch API call

In [4]:
def chunks(list, n):
    for i in range(0, len(list), n):
        yield list[i:i + n]

symbol_lists = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_lists)):
    symbol_strings.append(','.join(symbol_lists[i]))
symbol_strings

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

In [5]:
column_title = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

df = pd.DataFrame(columns=column_title)
df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy


In [6]:
for symbol_string in symbol_strings:
    batch_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,price&token={IEX_CLOUD_API_TOKEN}'
    data2 = requests.get(batch_url).json()
    for symbol in symbol_string.split(','):
        df = df.append(
            pd.Series(
                [
                    symbol,
                    data2[symbol]['price'],
                    data2[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                ],
                index=column_title),
            ignore_index=True)
df.head()

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,131.300,0.418156,N/A
1,AAL,16.592,-0.451884,N/A
2,AAP,168.080,0.106384,N/A
3,AAPL,129.300,0.613580,N/A
4,ABBV,112.900,0.326881,N/A


# Identify the 50 highest momentum change stocks

In [7]:
df.sort_values('One-Year Price Return', ascending=False, inplace=True)
df = df[:50]
df.reset_index(inplace=True)

In [8]:
len(df)

50

# Calculating the number of shares to buy

In [9]:
def portfolio_size():
    global capitals
    capitals = input('Please enter how much money you are going to invest: ')

    try:
        capitals = float(capitals)
    except ValueError:
        print("That's not a number.\nPlease try again.")
        capitals = input('Please enter how much money you are going to invest: ')
        capitals = float(capitals)

portfolio_size()
print(capitals)

10000000.0


In [10]:
weights = float(capitals / len(df))
weights

200000.0

In [11]:
df['Number of Shares to Buy'] = np.floor(weights / df['Price'])
df.head()

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,42.14,2.407000,4746.0
1,179,FCX,32.10,1.369067,6230.0
2,275,LB,46.41,1.329570,4309.0
3,23,ALB,186.21,1.270806,1074.0
4,387,PYPL,259.13,1.103910,771.0


## Distinguish the high quality momentum stocks and low quality momentum stocks
low quality momentum might be caused by short-term events that are not goign to repeat in the future

In [12]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    '1y Price Return',
    '1y Return Percentile',
    '6m Price Return',
    '6m Return Percentile',
    '3m Price Return',
    '3m Return Percentile',
    '1m Price Return',
    '1m Return Percentile'
]

df_hqm = pd.DataFrame(columns=hqm_columns)
df_hqm

,Ticker,Price,Number of Shares to Buy,1y Price Return,1y Return Percentile,6m Price Return,6m Return Percentile,3m Price Return,3m Return Percentile,1m Price Return,1m Return Percentile


In [13]:
for symbol_string in symbol_strings:
    batch_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats,price&token={IEX_CLOUD_API_TOKEN}'
    data2 = requests.get(batch_url).json()
    for symbol in symbol_string.split(','):
        df_hqm = df_hqm.append(
            pd.Series(
                [
                    symbol,
                    data2[symbol]['price'],
                    'N/A',
                    data2[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data2[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data2[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data2[symbol]['stats']['month1ChangePercent'],
                    'N/A'
                ],
                index=hqm_columns),
            ignore_index=True)

df_hqm

,Ticker,Price,Number of Shares to Buy,1y Price Return,1y Return Percentile,6m Price Return,6m Return Percentile,3m Price Return,3m Return Percentile,1m Price Return,1m Return Percentile
0,A,129.380,N/A,0.425021,N/A,0.345999,N/A,0.189159,N/A,0.061461,N/A
1,AAL,16.012,N/A,-0.446751,N/A,0.328102,N/A,0.269057,N/A,-0.045807,N/A
2,AAP,171.240,N/A,0.104651,N/A,0.150818,N/A,0.069595,N/A,0.032977,N/A
3,AAPL,133.190,N/A,0.616785,N/A,0.333535,N/A,0.071171,N/A,0.003957,N/A
4,ABBV,117.620,N/A,0.335536,N/A,0.129643,N/A,0.300872,N/A,0.072847,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.050,N/A,0.031133,N/A,0.188677,N/A,0.091060,N/A,-0.029194,N/A
501,ZBH,161.190,N/A,0.083396,N/A,0.192118,N/A,0.093084,N/A,0.057471,N/A
502,ZBRA,406.570,N/A,0.642184,N/A,0.516280,N/A,0.412547,N/A,0.099164,N/A
503,ZION,51.670,N/A,0.005175,N/A,0.540815,N/A,0.592084,N/A,0.200825,N/A


## Calculating the return percentile

In [14]:
df_hqm

,Ticker,Price,Number of Shares to Buy,1y Price Return,1y Return Percentile,6m Price Return,6m Return Percentile,3m Price Return,3m Return Percentile,1m Price Return,1m Return Percentile
0,A,129.380,N/A,0.425021,N/A,0.345999,N/A,0.189159,N/A,0.061461,N/A
1,AAL,16.012,N/A,-0.446751,N/A,0.328102,N/A,0.269057,N/A,-0.045807,N/A
2,AAP,171.240,N/A,0.104651,N/A,0.150818,N/A,0.069595,N/A,0.032977,N/A
3,AAPL,133.190,N/A,0.616785,N/A,0.333535,N/A,0.071171,N/A,0.003957,N/A
4,ABBV,117.620,N/A,0.335536,N/A,0.129643,N/A,0.300872,N/A,0.072847,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.050,N/A,0.031133,N/A,0.188677,N/A,0.091060,N/A,-0.029194,N/A
501,ZBH,161.190,N/A,0.083396,N/A,0.192118,N/A,0.093084,N/A,0.057471,N/A
502,ZBRA,406.570,N/A,0.642184,N/A,0.516280,N/A,0.412547,N/A,0.099164,N/A
503,ZION,51.670,N/A,0.005175,N/A,0.540815,N/A,0.592084,N/A,0.200825,N/A


In [35]:
time_periods = ['1y', '6m', '3m', '1m']

for row in df_hqm.index:
    for period in time_periods:
        return_change = f'{period} Price Return'
        return_percentile = f'{period} Return Percentile'
        df_hqm[return_percentile] = 0.0
        if df_hqm.loc[row, return_change] == None:
            df_hqm.loc[row, return_change] = 0.0

for row in df_hqm.index:
    for period in time_periods:
        return_change = f'{period} Price Return'
        return_percentile = f'{period} Return Percentile'
        df_hqm.loc[row, return_percentile] = stats.percentileofscore(df_hqm[return_change], df_hqm.loc[row, return_change]) / 100

df_hqm.head()

,Ticker,Price,Number of Shares to Buy,1y Price Return,1y Return Percentile,6m Price Return,6m Return Percentile,3m Price Return,3m Return Percentile,1m Price Return,1m Return Percentile,hqm_score
0,DISCA,36.41,549.0,0.134138,0.54,0.735230,0.66,0.776114,0.88,0.336713,1.00,100.000000
1,DISCK,33.31,600.0,0.062251,0.44,0.696595,0.58,0.739484,0.84,0.330427,0.98,99.801980
2,SIVB,473.28,42.0,0.813744,0.92,1.129548,0.92,0.713063,0.80,0.302764,0.96,99.603960
3,ALB,194.22,102.0,1.281740,0.98,1.094303,0.90,0.951490,0.92,0.301047,0.94,99.405941
4,VIAC,43.98,454.0,0.198863,0.66,0.852088,0.78,0.653421,0.76,0.296605,0.92,99.207921


## HQM Score
the mean of 4 percentile scores

In [36]:
df['hqm_score'] = 'N/A'

for row in df_hqm.index:
    for period in time_periods:
        hqm_percentile = []
        hqm_percentile.append(df_hqm.loc[row, return_percentile])

        df_hqm.loc[row, 'hqm_score'] = np.mean(hqm_percentile)

df_hqm

,Ticker,Price,Number of Shares to Buy,1y Price Return,1y Return Percentile,6m Price Return,6m Return Percentile,3m Price Return,3m Return Percentile,1m Price Return,1m Return Percentile,hqm_score
0,DISCA,36.410,549.0,0.134138,0.54,0.735230,0.66,0.776114,0.88,0.336713,1.00,1.00
1,DISCK,33.310,600.0,0.062251,0.44,0.696595,0.58,0.739484,0.84,0.330427,0.98,0.98
2,SIVB,473.280,42.0,0.813744,0.92,1.129548,0.92,0.713063,0.80,0.302764,0.96,0.96
3,ALB,194.220,102.0,1.281740,0.98,1.094303,0.90,0.951490,0.92,0.301047,0.94,0.94
4,VIAC,43.980,454.0,0.198863,0.66,0.852088,0.78,0.653421,0.76,0.296605,0.92,0.92
5,FANG,62.400,320.0,-0.320834,0.08,0.555564,0.44,1.057284,0.96,0.294074,0.90,0.90
6,FLIR,56.180,355.0,-0.001026,0.34,0.319879,0.10,0.463227,0.42,0.284724,0.88,0.88
7,DVN,21.000,952.0,-0.204656,0.12,0.995734,0.86,1.238318,0.98,0.262469,0.86,0.86
8,FTI,11.840,1689.0,-0.397532,0.04,0.547964,0.42,0.748908,0.86,0.250000,0.84,0.84
9,DXC,29.320,682.0,-0.188938,0.14,0.723030,0.64,0.397663,0.24,0.244105,0.82,0.82


In [37]:
# 50 best momentum stocks

df_hqm.sort_values('hqm_score', ascending=False, inplace=True)
df_hqm = df_hqm[:50]
df_hqm.reset_index(inplace=True, drop=True)
df_hqm

,Ticker,Price,Number of Shares to Buy,1y Price Return,1y Return Percentile,6m Price Return,6m Return Percentile,3m Price Return,3m Return Percentile,1m Price Return,1m Return Percentile,hqm_score
0,DISCA,36.410,549.0,0.134138,0.54,0.735230,0.66,0.776114,0.88,0.336713,1.00,1.00
1,DISCK,33.310,600.0,0.062251,0.44,0.696595,0.58,0.739484,0.84,0.330427,0.98,0.98
2,SIVB,473.280,42.0,0.813744,0.92,1.129548,0.92,0.713063,0.80,0.302764,0.96,0.96
3,ALB,194.220,102.0,1.281740,0.98,1.094303,0.90,0.951490,0.92,0.301047,0.94,0.94
4,VIAC,43.980,454.0,0.198863,0.66,0.852088,0.78,0.653421,0.76,0.296605,0.92,0.92
5,FANG,62.400,320.0,-0.320834,0.08,0.555564,0.44,1.057284,0.96,0.294074,0.90,0.90
6,FLIR,56.180,355.0,-0.001026,0.34,0.319879,0.10,0.463227,0.42,0.284724,0.88,0.88
7,DVN,21.000,952.0,-0.204656,0.12,0.995734,0.86,1.238318,0.98,0.262469,0.86,0.86
8,FTI,11.840,1689.0,-0.397532,0.04,0.547964,0.42,0.748908,0.86,0.250000,0.84,0.84
9,DXC,29.320,682.0,-0.188938,0.14,0.723030,0.64,0.397663,0.24,0.244105,0.82,0.82


In [38]:
portfolio_size()

In [39]:
weights2 = capitals / len(df_hqm)
weights2

20000.0

In [40]:
df_hqm['Number of Shares to Buy'] = np.floor(weights2 / df_hqm['Price'])
df_hqm.head()

,Ticker,Price,Number of Shares to Buy,1y Price Return,1y Return Percentile,6m Price Return,6m Return Percentile,3m Price Return,3m Return Percentile,1m Price Return,1m Return Percentile,hqm_score
0,DISCA,36.41,549.0,0.134138,0.54,0.735230,0.66,0.776114,0.88,0.336713,1.00,1.00
1,DISCK,33.31,600.0,0.062251,0.44,0.696595,0.58,0.739484,0.84,0.330427,0.98,0.98
2,SIVB,473.28,42.0,0.813744,0.92,1.129548,0.92,0.713063,0.80,0.302764,0.96,0.96
3,ALB,194.22,102.0,1.281740,0.98,1.094303,0.90,0.951490,0.92,0.301047,0.94,0.94
4,VIAC,43.98,454.0,0.198863,0.66,0.852088,0.78,0.653421,0.76,0.296605,0.92,0.92


In [41]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
df_hqm.to_excel(writer, sheet_name='Monmentum Strategy', index=False)

In [42]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

percentage_format = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [43]:
columns_format = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Number of Shares to Buy', integer_format],
    'D': ['1y Price Return', percentage_format],
    'E': ['1y Return Percentile', percentage_format],
    'F': ['6m Price Return', percentage_format],
    'G': ['6m Return Percentile', percentage_format],
    'H': ['3m Price Return', percentage_format],
    'I': ['3m Return Percentile', percentage_format],
    'J': ['1m Price Return', percentage_format],
    'K': ['1m Return Percentile', percentage_format],
    'L': ['HQM Score', percentage_format]
}

for column in columns_format:
    writer.sheets['Monmentum Strategy'].set_column(f'{column}:{column}', 18, columns_format[column][1])
writer.save()